# Pipeline de consulta sobre Grafo de Conocimiento (NLP + ML)

Este cuaderno implementa un flujo práctico **Query → Selection Function → Planner** para consultar un grafo de conocimiento con técnicas de NLP, embeddings y algoritmos de búsqueda en grafos.

**Dependencias principales:** networkx, numpy, scikit-learn, sentence-transformers, spaCy, matplotlib.

> Nota: el cuaderno intenta instalar paquetes faltantes y descargar el modelo de spaCy automáticamente si es necesario.

## 1. Import Required Libraries

Importamos librerías para grafos, NLP y embeddings.

In [13]:
# Instalación ligera si faltan paquetes
import sys
import importlib
import subprocess


def _ensure_package(pkg, import_name=None):
    name = import_name or pkg
    try:
        importlib.import_module(name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])


for pkg, imp in [
    ("networkx", None),
    ("numpy", None),
    ("scikit-learn", "sklearn"),
    ("sentence-transformers", "sentence_transformers"),
    ("spacy", None),
    ("matplotlib", "matplotlib"),
    ("plotly", None),
]:
    _ensure_package(pkg, imp)

# Imports principales
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import spacy
from spacy.cli import download as spacy_download
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Cargar modelo spaCy para NER en español
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    print("Descargando modelo de spaCy en español...")
    spacy_download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

# Cargar modelo de embeddings multilingüe
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

print("Dependencias listas.")

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 795.92it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Dependencias listas.


## 2. Load and Build Knowledge Graph

Creamos un grafo de conocimiento de ejemplo con entidades y relaciones semánticas.

In [2]:
# Grafo dirigido para representar relaciones
G = nx.DiGraph()

# Nodos con atributos semánticos en español
nodes = {
    "Tesla": {"type": "Empresa", "description": "Fabricante de vehículos eléctricos y empresa de energía"},
    "Elon Musk": {"type": "Persona", "description": "Emprendedor y CEO asociado con Tesla y SpaceX"},
    "SpaceX": {"type": "Empresa", "description": "Fabricante aeroespacial y servicios de transporte espacial"},
    "Batería": {"type": "Tecnología", "description": "Tecnología de almacenamiento de energía para vehículos eléctricos"},
    "Autopilot": {"type": "Producto", "description": "Sistema de asistencia al conductor para conducción autónoma"},
    "Energía Solar": {"type": "Dominio", "description": "Energía renovable proveniente del sol"},
    "Mercado VE": {"type": "Mercado", "description": "Mercado de vehículos eléctricos e industria relacionada"},
    "Estados Unidos": {"type": "Ubicación", "description": "País donde se fundó Tesla"},
    "Lanzamiento": {"type": "Evento", "description": "Evento de lanzamiento de un nuevo producto"},
    "Starlink": {"type": "Producto", "description": "Constelación de satélites de internet por SpaceX"},
    "IA": {"type": "Tecnología", "description": "Inteligencia artificial y aprendizaje automático"},
    "Cibertruck": {"type": "Producto", "description": "Camioneta eléctrica de Tesla"},
}

for node, attrs in nodes.items():
    G.add_node(node, **attrs)

# Aristas con relaciones en español
edges = [
    ("Elon Musk", "Tesla", {"relation": "dirige", "description": "CEO de"}),
    ("Elon Musk", "SpaceX", {"relation": "fundó", "description": "Fundador de"}),
    ("Tesla", "Batería", {"relation": "utiliza", "description": "Usa almacenamiento de energía"}),
    ("Tesla", "Autopilot", {"relation": "desarrolla", "description": "Desarrolla asistencia al conductor"}),
    ("Tesla", "Energía Solar", {"relation": "invierte_en", "description": "Invierte en energía renovable"}),
    ("Tesla", "Mercado VE", {"relation": "compite_en", "description": "Compite en el mercado"}),
    ("Tesla", "Estados Unidos", {"relation": "ubicada_en", "description": "Sede en"}),
    ("Tesla", "Cibertruck", {"relation": "produce", "description": "Fabrica el vehículo"}),
    ("SpaceX", "Lanzamiento", {"relation": "realiza", "description": "Realiza lanzamientos"}),
    ("SpaceX", "Starlink", {"relation": "opera", "description": "Opera la red de satélites"}),
    ("Starlink", "Estados Unidos", {"relation": "regulado_en", "description": "Regulado en"}),
    ("Autopilot", "IA", {"relation": "usa", "description": "Utiliza tecnología de"}),
    ("Cibertruck", "Batería", {"relation": "equipado_con", "description": "Equipado con"}),
]

for src, dst, attrs in edges:
    # Peso base opcional (se puede ajustar con heurísticas)
    attrs.setdefault("weight", 1.0)
    G.add_edge(src, dst, **attrs)

print(f"Grafo creado con {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas.")

Grafo creado con 12 nodos y 13 aristas.


## 3. Query Processing with NLP

Procesamos la consulta del usuario: tokenización, NER y embeddings de texto.

In [4]:
def process_query(query: str):
    """Tokenización, NER y embedding de la consulta."""
    doc = nlp(query)
    tokens = [t.text for t in doc]
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    embedding = embedder.encode(query)
    return {
        "query": query,
        "tokens": tokens,
        "entities": entities,
        "embedding": embedding,
    }


# Demo rápida (se puede cambiar luego)
_demo_query = "¿Qué empresas dirige Elon Musk relacionadas con vehículos eléctricos?"
_q = process_query(_demo_query)

print("Consulta original:", _q["query"])
print("Tokens:", _q["tokens"])
print("Entidades identificadas:", _q["entities"])
print("Vector (primeras 10 dims):", _q["embedding"][:10])
print("Dimensión del embedding:", _q["embedding"].shape)

Consulta original: ¿Qué empresas dirige Elon Musk relacionadas con vehículos eléctricos?
Tokens: ['¿', 'Qué', 'empresas', 'dirige', 'Elon', 'Musk', 'relacionadas', 'con', 'vehículos', 'eléctricos', '?']
Entidades identificadas: [('Elon Musk', 'PER')]
Vector (primeras 10 dims): [-0.08681609  0.18746856  0.10550842  0.08222028 -0.13728622 -0.09994306
  0.40185967  0.01190598 -0.02209243 -0.40775773]
Dimensión del embedding: (384,)


## 4. Selection Function: Entity Matching

Comparamos el embedding de la consulta con los embeddings de los nodos del grafo.

In [5]:
def node_text(node, attrs):
    return f"{node} ({attrs.get('type')}): {attrs.get('description')}"

# Precomputar embeddings de nodos
node_embeddings = {}
for node, attrs in G.nodes(data=True):
    text = node_text(node, attrs)
    node_embeddings[node] = embedder.encode(text)


def select_relevant_entities(query_embedding, top_k=3):
    """Calcula similitud entre consulta y nodos del grafo."""
    scores = {}
    for node, emb in node_embeddings.items():
        score = float(cosine_similarity([query_embedding], [emb])[0][0])
        scores[node] = score
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_k], ranked


_top_candidates, _all_ranked = select_relevant_entities(_q["embedding"], top_k=3)
print("Entidades candidatas (top-k):", _top_candidates)

Entidades candidatas (top-k): [('Elon Musk', 0.7248793840408325), ('Tesla', 0.6873642206192017), ('Cibertruck', 0.6071425676345825)]


## 5. Similarity Computation and Ranking

Calculamos y mostramos puntajes de relevancia para entidades candidatas y relaciones cercanas.

In [8]:
def rank_entities(query_embedding, top_k=5):
    top, ranked = select_relevant_entities(query_embedding, top_k=top_k)
    return top, ranked


def candidate_relations(top_entities):
    """Relaciones conectadas a las entidades top con puntaje aproximado."""
    rels = []
    scores = dict(top_entities)
    for node, score in top_entities:
        for _, dst, attrs in G.out_edges(node, data=True):
            rels.append((node, attrs.get("relation"), dst, score))
    return rels


_top, _ranked = rank_entities(_q["embedding"], top_k=5)
print("Ranking (top-5):")
for ent, score in _top:
    print(f"  - {ent}: {score:.4f}")

print("\nRelaciones candidatas:")
for src, rel, dst, score in candidate_relations(_top):
    print(f"  - {src} --{rel}--> {dst} (score {score:.4f})")

Ranking (top-5):
  - Elon Musk: 0.7249
  - Tesla: 0.6874
  - Cibertruck: 0.6071
  - Mercado VE: 0.5224
  - Batería: 0.4932

Relaciones candidatas:
  - Elon Musk --dirige--> Tesla (score 0.7249)
  - Elon Musk --fundó--> SpaceX (score 0.7249)
  - Tesla --utiliza--> Batería (score 0.6874)
  - Tesla --desarrolla--> Autopilot (score 0.6874)
  - Tesla --invierte_en--> Energía Solar (score 0.6874)
  - Tesla --compite_en--> Mercado VE (score 0.6874)
  - Tesla --ubicada_en--> Estados Unidos (score 0.6874)
  - Tesla --produce--> Cibertruck (score 0.6874)
  - Cibertruck --equipado_con--> Batería (score 0.6071)


## 6. Planner: Path Finding Algorithms

Aplicamos BFS, DFS y camino más corto entre entidades relevantes.

In [33]:
from collections import deque


def bfs_paths(graph, start, goal, max_depth=4):
    """Búsqueda BFS para caminos simples (considera grafo no dirigido)."""
    G_undirected = graph.to_undirected()
    paths = []
    queue = deque([(start, [start])])
    while queue:
        node, path = queue.popleft()
        if len(path) > max_depth:
            continue
        if node == goal:
            paths.append(path)
            continue
        for neighbor in G_undirected.neighbors(node):
            if neighbor not in path:
                queue.append((neighbor, path + [neighbor]))
    return paths


def dfs_paths(graph, start, goal, max_depth=4):
    """Búsqueda DFS para caminos simples (considera grafo no dirigido)."""
    G_undirected = graph.to_undirected()
    paths = []

    def _dfs(node, path):
        if len(path) > max_depth:
            return
        if node == goal:
            paths.append(path)
            return
        for neighbor in G_undirected.neighbors(node):
            if neighbor not in path:
                _dfs(neighbor, path + [neighbor])

    _dfs(start, [start])
    return paths


def shortest_path(graph, start, goal):
    """Camino más corto (considera grafo no dirigido)."""
    try:
        G_undirected = graph.to_undirected()
        return nx.shortest_path(G_undirected, start, goal, weight="weight")
    except nx.NetworkXNoPath:
        return None


def path_score(path, scores):
    if not path:
        return 0.0
    base = np.mean([scores.get(n, 0.0) for n in path])
    length_penalty = 1.0 / len(path)
    return float(base + length_penalty)


# Demo con las dos entidades más relevantes
if len(_top) >= 2:
    src = _top[0][0]
    dst = _top[1][0]
    bfs = bfs_paths(G, src, dst, max_depth=4)
    dfs = dfs_paths(G, src, dst, max_depth=4)
    sp = shortest_path(G, src, dst)

    print("BFS paths:", bfs)
    print("DFS paths:", dfs)
    print("Shortest path:", sp)
else:
    print("No hay suficientes entidades para planificar rutas.")

BFS paths: [['Elon Musk', 'Tesla']]
DFS paths: [['Elon Musk', 'Tesla']]
Shortest path: ['Elon Musk', 'Tesla']


## 7. Path Interpretation and Response Generation

Interpretamos los caminos y generamos una respuesta razonada.

In [38]:
def path_to_text(graph, path):
    if not path or len(path) == 1:
        return path[0] if path else ""
    parts = []
    for i in range(len(path) - 1):
        src = path[i]
        dst = path[i + 1]
        # Verificar ambas direcciones
        if graph.has_edge(src, dst):
            rel = graph.edges[src, dst].get("relation", "related_to")
            parts.append(f"{src} --{rel}--> {dst}")
        elif graph.has_edge(dst, src):
            rel = graph.edges[dst, src].get("relation", "related_to")
            parts.append(f"{src} <--{rel}-- {dst}")
        else:
            parts.append(f"{src} --- {dst}")
    return " | ".join(parts)


def generate_answer(query, top_entities, paths, scores):
    if not paths:
        return "No se encontró información relevante en el grafo."
    
    scored_paths = [(p, path_score(p, scores)) for p in paths]
    scored_paths.sort(key=lambda x: x[1], reverse=True)
    best_path, best_score = scored_paths[0]
    
    # Generar respuesta en lenguaje natural basada en el camino
    if len(best_path) >= 2:
        entities_in_path = " → ".join(best_path)
        return f"Sí. El camino encontrado es: {entities_in_path}"
    else:
        return f"Entidad relevante: {best_path[0] if best_path else 'ninguna'}"


# Demo con rutas BFS si existen
scores_dict = dict(_ranked)
if len(_top) >= 2:
    src = _top[0][0]
    dst = _top[1][0]
    paths_demo = bfs_paths(G, src, dst, max_depth=4)
    print(generate_answer(_q["query"], _top, paths_demo, scores_dict))

Sí. El camino encontrado es: Elon Musk → Tesla


## 8. End-to-End Pipeline Execution

Orquestamos todas las etapas en una función única y ejecutamos una consulta.

In [43]:
def run_pipeline(query, top_k=3, max_depth=4, show_viz=False):
    """
    Ejecuta el pipeline completo para una consulta.
    
    Args:
        query: Consulta en lenguaje natural
        top_k: Número de entidades más relevantes a seleccionar
        max_depth: Profundidad máxima para búsqueda de caminos
        show_viz: Si True, muestra la visualización del grafo
    
    Returns:
        dict con resultados del pipeline
    """
    print("\n=== QUERY PROCESSING ===")
    q = process_query(query)
    print(f"Consulta: {q['query']}")
    print(f"Entidades NER: {q['entities']}")

    print("\n=== SELECTION FUNCTION ===")
    top_entities, ranked = select_relevant_entities(q["embedding"], top_k=top_k)
    print("Entidades relevantes:")
    for i, (ent, score) in enumerate(top_entities, 1):
        print(f"  {i}. {ent}: {score:.4f}")

    print("\n=== PLANNER ===")
    scores_dict = dict(ranked)
    paths = []
    if len(top_entities) >= 2:
        src, dst = top_entities[0][0], top_entities[1][0]
        print(f"Caminos: {src} -> {dst}")
        bfs = bfs_paths(G, src, dst, max_depth=max_depth)
        dfs = dfs_paths(G, src, dst, max_depth=max_depth)
        sp = shortest_path(G, src, dst)
        print(f"  BFS: {len(bfs)} | DFS: {len(dfs)} | Mas corto: {sp if sp else 'N/A'}")
        paths = bfs if bfs else ([] if sp is None else [sp])
    else:
        print("Entidades insuficientes para rutas.")

    print("\n=== RESPUESTA ===")
    answer = generate_answer(q["query"], top_entities, paths, scores_dict)
    print(answer)
    print()

    result = {
        "query": q,
        "top_entities": top_entities,
        "ranked": ranked,
        "paths": paths,
    }
    
    if show_viz:
        visualize_graph_interactive(G, top_entities=top_entities, paths=paths)
    
    return result


# Ingrese su consulta
try:
    user_query = input("Ingrese query: ").strip()
except Exception:
    user_query = ""

if not user_query:
    user_query = "¿Cual es la relacion entre Elon Musk y Tesla en el mercado de vehiculos electricos?"

results = run_pipeline(user_query, top_k=3, max_depth=4, show_viz=False)


=== QUERY PROCESSING ===
Consulta: ¿Qué relación hay entre Tesla y el Autopilot?
Entidades NER: [('Qué', 'PER'), ('Tesla', 'LOC'), ('Autopilot', 'LOC')]

=== SELECTION FUNCTION ===
Entidades relevantes:
  1. Autopilot: 0.6232
  2. Tesla: 0.6054
  3. Cibertruck: 0.5952

=== PLANNER ===
Caminos: Autopilot -> Tesla
  BFS: 1 | DFS: 1 | Mas corto: ['Autopilot', 'Tesla']

=== RESPUESTA ===
Sí. El camino encontrado es: Autopilot → Tesla



## 9. Visualización Interactiva del Grafo

Visualiza el grafo con las entidades y caminos encontrados.

In [44]:
def visualize_graph_interactive(graph, top_entities=None, paths=None):
    """Visualización interactiva del grafo con Plotly."""
    pos = nx.spring_layout(graph, seed=42, k=0.5)
    
    # Preparar datos de aristas
    edge_trace = []
    edge_labels = []
    
    for u, v, data in graph.edges(data=True):
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        
        # Determinar color de la arista
        edge_color = "#888"
        edge_width = 1
        
        # Resaltar aristas en los caminos
        if paths:
            for path in paths:
                path_edges = list(zip(path[:-1], path[1:]))
                if (u, v) in path_edges:
                    edge_color = "#4E79A7"
                    edge_width = 3
                    break
        
        edge_trace.append(
            go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                mode='lines',
                line=dict(width=edge_width, color=edge_color),
                hoverinfo='text',
                text=f"{u} --{data.get('relation', 'rel')}--> {v}",
                showlegend=False
            )
        )
        
        # Añadir etiqueta en el medio de la arista
        edge_labels.append(
            go.Scatter(
                x=[(x0 + x1) / 2],
                y=[(y0 + y1) / 2],
                mode='text',
                text=data.get('relation', ''),
                textfont=dict(size=9, color='#555'),
                hoverinfo='none',
                showlegend=False
            )
        )
    
    # Preparar datos de nodos
    top_set = {n for n, _ in (top_entities or [])}
    
    node_x = []
    node_y = []
    node_text = []
    node_colors = []
    node_sizes = []
    
    for node in graph.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        
        attrs = graph.nodes[node]
        hover_text = f"<b>{node}</b><br>Tipo: {attrs.get('type')}<br>{attrs.get('description')}"
        node_text.append(hover_text)
        
        if node in top_set:
            node_colors.append('#FF6F61')
            node_sizes.append(25)
        else:
            node_colors.append('#A0A0A0')
            node_sizes.append(20)
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=[node for node in graph.nodes()],
        textposition="top center",
        textfont=dict(size=10, color='black'),
        hovertext=node_text,
        hoverinfo='text',
        marker=dict(
            size=node_sizes,
            color=node_colors,
            line=dict(width=2, color='white')
        ),
        showlegend=False
    )
    
    # Crear figura
    fig = go.Figure(data=edge_trace + edge_labels + [node_trace])
    
    fig.update_layout(
        title=dict(
            text="Grafo de Conocimiento Interactivo",
            x=0.5,
            xanchor='center',
            font=dict(size=18)
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=20, r=20, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='white',
        height=700
    )
    
    fig.show()


# Visualizar el grafo con los resultados de la ultima consulta
if results is not None:
    print("Generando visualizacion interactiva del grafo...")
    visualize_graph_interactive(G, top_entities=results.get("top_entities"), paths=results.get("paths"))
else:
    print("No hay resultados previos. Ejecute primero una consulta en la celda anterior.")

Generando visualizacion interactiva del grafo...


## Verificación del flujo del agente (LangChain + LangGraph)
Este bloque muestra logs secuenciales, embeddings explícitos y un plan de ejecución con LangGraph.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langgraph.graph import StateGraph, END
from sklearn.metrics.pairwise import cosine_similarity

# Input
query = "hola, ¿cómo estás?"
print(f"input: {query}")

# Embeddings (modelo explícito)
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
emb = HuggingFaceEmbeddings(model_name=model_name)
print(f"embedding_model: {model_name}")
q_emb = emb.embed_query(query)

# Function selection (similitud)
funciones = {
    "smalltalk": "Saludo y estado del asistente",
    "buscar_producto": "Busca productos por nombre o categoria",
    "verificar_stock": "Consulta disponibilidad de productos",
}
func_embs = {k: emb.embed_query(v) for k, v in funciones.items()}
scored = sorted([(k, float(cosine_similarity([q_emb], [v])[0][0])) for k, v in func_embs.items()], key=lambda x: x[1], reverse=True)
selected = scored[0][0]
print(f"seleccion_funcion: {selected}")

# Plan con LangGraph (explícito y secuencial)
plan = ["explorar_grafo", selected]
print(f"plan: {plan}")

state = {"query": query, "logs": []}

graph = StateGraph(dict)

def step_explorar_grafo(s):
    s["logs"].append("step: explorar_grafo")
    print("step: explorar_grafo")
    return s

def step_smalltalk(s):
    s["logs"].append("step: smalltalk")
    print("step: smalltalk")
    return s

def step_buscar_producto(s):
    s["logs"].append("step: buscar_producto")
    print("step: buscar_producto")
    return s

def step_verificar_stock(s):
    s["logs"].append("step: verificar_stock")
    print("step: verificar_stock")
    return s

node_map = {
    "explorar_grafo": step_explorar_grafo,
    "smalltalk": step_smalltalk,
    "buscar_producto": step_buscar_producto,
    "verificar_stock": step_verificar_stock,
}

prev = None
for i, step in enumerate(plan):
    name = f"step_{i}_{step}"
    graph.add_node(name, node_map.get(step, step_smalltalk))
    if prev is None:
        graph.set_entry_point(name)
    else:
        graph.add_edge(prev, name)
    prev = name

graph.add_edge(prev, END)
app = graph.compile()
app.invoke(state)

print("output: respuesta natural generada")